In [1]:
import pandas as pd
import numpy as np

# Leer el archivo CSV
df = pd.read_csv('data/edges.csv')

# Obtener la lista única de nodos
nodos = pd.unique(df[['Node 1', 'Node 2']].values.ravel())

# Crear un DataFrame vacío con nodos como índices y columnas
matriz_similitud = pd.DataFrame(np.nan, index=nodos, columns=nodos)

# Rellenar la matriz con los valores de Cosine Similarity
for _, row in df.iterrows():
    nodo1 = row['Node 1']
    nodo2 = row['Node 2']
    similitud = row['Cosine Similarity']
    matriz_similitud.at[nodo1, nodo2] = similitud
    matriz_similitud.at[nodo2, nodo1] = similitud  # Si la matriz es simétrica

# Opcional: Reemplazar NaN con 0 si no hay similitud
matriz_similitud.fillna(0, inplace=True)

# Mostrar la matriz
print(matriz_similitud)

          0         843       1         89   2         119       137  \
0    0.000000  0.306233  0.000000  0.000000  0.0  0.000000  0.000000   
843  0.306233  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
1    0.000000  0.000000  0.000000  0.323714  0.0  0.000000  0.000000   
89   0.000000  0.000000  0.323714  0.000000  0.0  0.000000  0.000000   
2    0.000000  0.000000  0.000000  0.000000  0.0  0.340926  0.333013   
..        ...       ...       ...       ...  ...       ...       ...   
744  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
801  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
759  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
814  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
761  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   

          139       714  439  ...  804  727  844  800  755  744  801  \
0    0.000000  0.000000  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0

In [5]:
import pandas as pd

# Paso 1: Leer los archivos CSV
targets_df = pd.read_csv('data/targets_con_palabras_clave_agrupadas.csv')
nodes_df = pd.read_csv('data/nodes.csv')

# Mostrar los DataFrames leídos (opcional)
print("DataFrame de Targets:")
print(targets_df)
print("\nDataFrame de Nodes:")
print(nodes_df)

# Paso 2: Crear un diccionario para mapear palabras a Node IDs
# Convertir las palabras a minúsculas para asegurar una correspondencia insensible a mayúsculas
nodes_dict = dict(zip(nodes_df['Word'].str.lower(), nodes_df['Node ID']))

print("\nDiccionario de Mapeo (Word -> Node ID):")
print(nodes_dict)

# Inicializar un conjunto para almacenar las palabras clave no encontradas
missing_keywords = set()

# Paso 3: Definir una función para mapear palabras clave a Node IDs
def map_keywords_to_node_ids(keywords, mapping_dict, missing_set):
    """
    Toma una cadena de palabras clave separadas por comas y devuelve una lista de Node IDs correspondientes.
    Añade las palabras no encontradas al conjunto missing_set.
    
    Parámetros:
    - keywords (str): Cadena de palabras clave separadas por comas.
    - mapping_dict (dict): Diccionario que mapea palabras a Node IDs.
    - missing_set (set): Conjunto para almacenar palabras no encontradas.
    
    Retorna:
    - list: Lista de Node IDs correspondientes a las palabras clave encontradas.
    """
    if pd.isna(keywords):
        return []
    
    # Dividir las palabras clave por comas y eliminar espacios en blanco
    words = [word.strip().lower() for word in keywords.split(',')]
    
    # Mapear cada palabra a su Node ID si existe en el diccionario
    node_ids = []
    for word in words:
        if word in mapping_dict:
            node_ids.append(mapping_dict[word])
        else:
            missing_set.add(word)
    
    return node_ids

# Paso 4: Aplicar la función al DataFrame de Targets para crear la nueva columna
targets_df['Node_IDs'] = targets_df['palabras_clave_concatenadas'].apply(
    lambda x: map_keywords_to_node_ids(x, nodes_dict, missing_keywords)
)

# Paso 5: Mostrar el DataFrame resultante
print("\nDataFrame Final con Node_IDs:")
print(targets_df)

# Paso 6: Guardar el DataFrame resultante en un archivo CSV (opcional)
targets_df.to_csv('targets_with_node_ids.csv', index=False)
print("\nEl DataFrame con Node_IDs ha sido guardado en 'targets_with_node_ids.csv'")

# Paso 7: Mostrar y/o guardar la lista de palabras clave no encontradas
if missing_keywords:
    print("\nPalabras clave no encontradas en 'nodes.csv':")
    for keyword in sorted(missing_keywords):
        print(f"- {keyword}")
    
    # Guardar las palabras clave faltantes en un archivo de texto
    with open('missing_keywords.txt', 'w', encoding='utf-8') as f:
        for keyword in sorted(missing_keywords):
            f.write(f"{keyword}\n")
    print("\nLa lista de palabras clave no encontradas ha sido guardada en 'missing_keywords.txt'")
else:
    print("\nTodas las palabras clave fueron encontradas en 'nodes.csv'.")


DataFrame de Targets:
     goal_id target_id                                             target  \
0          1       1.1  By 2030, eradicate extreme poverty for all peo...   
1          1       1.2  By 2030, reduce at least by half the proportio...   
2          1       1.3  Implement nationally appropriate social protec...   
3          1       1.4  By 2030, ensure that all men and women, in par...   
4          1       1.5  By 2030, build the resilience of the poor and ...   
..       ...       ...                                                ...   
164       17     17.15  Respect each country’s policy space and leader...   
165       17     17.16  Enhance the global partnership for sustainable...   
166       17     17.17  Encourage and promote effective public, public...   
167       17     17.18  By 2020, enhance capacity-building support to ...   
168       17     17.19  By 2030, build on existing initiatives to deve...   

                           palabras_clave_concatenada